# 1. Scraping qdrops.online with BeautifulSoup and parsing its content
## 1.1 capturing
the text we want is contained within this tag hiearcharchy: 
- div message -> text -> string if not None & p 
- div message -> div op -> string 
- div message -> abbr title, abbr.text
- meta lar -> span: time, name, source, num

### prematurely calling text or get_text() on div_text will render unnecessary text, text we'd later have to clean

### tags to extract():
- hyperlinks: https?\S+\b, www, twitter, instagram, etc (inevitably will have to regex)
- a href
- figure
- figcaption
- images
- div op containing no text or string
- replace punct with a single space, then replace spaces longer than 1 space with a single space
- it also might make things easier tokenizing them before hand
### cleaning 
- sub hyperlinks
- lower text
- split text 
- sub punctuation
- sub digits 
- join split words back into string
- append strings to list if strings 
- return list of cleaned strings 
## Recurring Problems
### inconsistent tag use: br, p, text, abbr 
- many more tags could have been abbreviated or propertied with its value
- pickling exceeds maximum recursion; solved by sys.get and set a higher recursion limit
- runtime of requests is > 1 min: solved by loading the pickled object
- unwanted text from hyperlinks, figcaptions, etc; solved by using BeautifulSoup extract() on unwanted objects

In [1]:
import requests
import nltk
import os, sys
import itertools
import re, string
import pandas as pd
import pickle
import timeit

from string import punctuation, digits
from collections import Counter
from bs4 import BeautifulSoup, NavigableString, Tag
from string import punctuation, digits
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
from nltk import RegexpParser, Tree
from nltk.util import ngrams

punctuation += str('’‘–…“”')
pd.set_option('display.max_colwidth', None)

We don't have to execute the following cell, we can skip to opening the pickled object

In [3]:
'''%%time
base_url = 'https://qposts.online/page/' 
urls = [base_url+str(i) for i in range(1, 105)]
page_requests = [requests.get(url) for url in urls]
soups = [BeautifulSoup(page.text, 'html.parser') for page in page_requests]
messages_original = [soups[i].findAll('div', 'message') for i in range(0, len(soups))]
messages_flat = list(itertools.chain.from_iterable(messages))
meta_lar = [soups[i].findAll('div', 'meta lar') for i in range(0, len(soups))]'''

"%%time\nbase_url = 'https://qposts.online/page/' \nurls = [base_url+str(i) for i in range(1, 105)]\npage_requests = [requests.get(url) for url in urls]\nsoups = [BeautifulSoup(page.text, 'html.parser') for page in page_requests]\nmessages_original = [soups[i].findAll('div', 'message') for i in range(0, len(soups))]\nmessages_flat = list(itertools.chain.from_iterable(messages))\nmeta_lar = [soups[i].findAll('div', 'meta lar') for i in range(0, len(soups))]"

In [2]:
os.getcwd()

'/Users/kylereaves/Documents/GitHub/parsing_madness'

In [3]:
index_df = pd.read_pickle('index_df.pkl')
index_df.number = index_df.number.astype('int')
date = [index_df.datetime[i].date() for i in range(0, len(index_df.datetime))]
time = [index_df.datetime[i].time() for i in range(0, len(index_df.datetime))]
dt_index = pd.MultiIndex.from_arrays([date, time])

In [4]:
%%time 
with open('messages_flat.pkl', 'rb') as f:
    messages = pickle.load(f)
with open('meta_flat.pkl', 'rb') as f:
    meta_lar = pickle.load(f)

CPU times: user 11.6 s, sys: 345 ms, total: 12 s
Wall time: 12 s


In [5]:
%%time
with open('names_joined.pkl', 'rb') as f:
    names = pickle.load(f)
with open('sources_joined.pkl', 'rb') as f:
    sources = pickle.load(f)

CPU times: user 1.66 ms, sys: 1.34 ms, total: 3 ms
Wall time: 8 ms


In [45]:
class Messages(object):
    def __init__(self):
        pass

    def get(integer: int):
        msg_list = []

        for item in messages[integer]:
            
            if isinstance(item, NavigableString) and item.name is None:
                msg_list.append(item)
            
            if isinstance(item, Tag) and item.name == 'p':
                msg_list.append(item.string)
            
            if isinstance(item, Tag) and item.name == 'div' and item.attrs == {'class': ['text']}:
                msg_list.append(item.text)
            
            for div_images in messages[integer].findAll('div', class_='images'):
                div_images.extract()
            for a_ref in messages[integer].findAll('a', class_='ref'):
                a_ref.extract()
            for a_href in messages[integer].findAll('a', class_='href'):
                a_ref.extract()
            for empty_line in messages[integer].findAll('p', class_='body-line empty'):
                empty_line.extract()
            for br_tag in messages[integer].findAll('br'):
                br_tag.replace_with(' ')

        cleaned = [item for item in msg_list if item != ' ' and item is not None]
        return cleaned

    def dataframe(integer: int):
        df = pd.DataFrame({'type': [type(i) for i in messages[integer].div.children],
                           'name': [i.name for i in messages[integer].div.children],
                           'content': [i for i in messages[integer].div.children]})
        return df

    def get_abbr(integer:int): 
        return Messages.dataframe(integer)[Messages.dataframe(integer).name == 'abbr']
    
    def info(integer:int):
        return index_df[index_df.number == integer]

    def sents(integer: int):
        return nltk.sent_tokenize(' '.join(Messages.get(integer)))
    
    def joined(integer: int):
        return ' '.join(Messages.get(integer))

    def split(integer: int):
        return Messages.joined(integer).split()

In [46]:
class Spans:
    def __init__(self):
        pass

    def nums():
        nums = [meta_lar[i].find('span', 'num').get_text() for i in range(0, len(meta_lar))]
        return nums
                
    def sources():
        sources = [meta_lar[i].find('span', 'source').get_text() for i in range(0, len(meta_lar))]
        links = [meta_lar[i].find('span', 'source').contents[-1].get('href') for i in range(0, len(meta_lar))]
        return sources
    
    def names():
        names = [meta_lar[i].find('span', 'name').get_text() for i in range(0, len(meta_lar))]
        return names
                      
    def dates():
        date_list = [meta_lar[i].find('span', 'time').get_text()for i in range(0, len(meta_lar))]
        dt_idx = pd.to_datetime(date_list, origin='unix', unit='s')
        return dt_idx

In [8]:
# if an item in Messages.split(i) is in replace_dict.keys(), replace i (the key) with its value
replace_dict = {'r v d': 'republicans vs democrats',
 'rs': 'republicans',
 "r's": 'republicans',
 "d's": 'democrats',
 'ds': 'democrats',
 '[D]': 'Democratic',
 'US': 'United States',
 ' w ': 'with',
 'w/': 'with',
 '&': 'and',
 'MSM': 'mainstream media',
 'ID': 'identification',
 'SA': 'Saudi Arabia',
 'MS-13': 'ms thirteen',
 'COVID19': 'covid',
 "M's": 'marshalls',
 'HUSSEIN': 'Barack Obama',
 'BRENNAN': 'John Brennan',
 'MERKEL': 'Angela Merkel',
 'KERRY': 'John Kerry',
 'v2': 'version two',
 'U.S.': 'United States',
 "Gov't": 'government'}

### using string with p tags saves us subing hyperlinks
### add a conditional statement on the end to pop None from list

In [76]:
[i.string for i in messages[10].div if i.string is not None]

['Fact checkers created in effort to reinforce propaganda [digestion]?',
 'The battle to prevent truth from reaching the people.',
 'The battle to maintain and push division.',
 'Divided you are weak.',
 'Divided you fight each other.',
 'Divided you pose no threat.',
 'System of control.',
 'Information warfare.',
 'Q']

In [45]:
Messages.dataframe(10)

,type,name,content
0,<class 'bs4.element.Tag'>,p,"[https:, [//], twitter.com/BrentScher/status/1322015793593360384]"
1,<class 'bs4.element.Tag'>,p,[Fact checkers created in effort to reinforce propaganda [digestion]?]
2,<class 'bs4.element.Tag'>,p,[The battle to prevent truth from reaching the people.]
3,<class 'bs4.element.Tag'>,p,[The battle to maintain and push division.]
4,<class 'bs4.element.Tag'>,p,[Divided you are weak.]
5,<class 'bs4.element.Tag'>,p,[Divided you fight each other.]
6,<class 'bs4.element.Tag'>,p,[Divided you pose no threat.]
7,<class 'bs4.element.Tag'>,p,[System of control.]
8,<class 'bs4.element.Tag'>,p,[Information warfare.]
9,<class 'bs4.element.Tag'>,p,[Q]


In [60]:
Messages.dataframe(3005)

,type,name,content
0,<class 'bs4.element.NavigableString'>,None,https:
1,<class 'bs4.element.Tag'>,em,[//]
2,<class 'bs4.element.NavigableString'>,None,in.reuters.com/article/iran-economy-rouhani-sanctions/iran-parliament-censures-rouhani-in-sign-pragmatists-losing-sway-idINKCN1LD0DL
3,<class 'bs4.element.NavigableString'>,None,
4,<class 'bs4.element.NavigableString'>,None,[Hassan Rouhani]
...,...,...,...
127,<class 'bs4.element.NavigableString'>,None,
128,<class 'bs4.element.NavigableString'>,None,Nothing to See Here.
129,<class 'bs4.element.NavigableString'>,None,
130,<class 'bs4.element.NavigableString'>,None,Q


In [64]:
[i.text for i in messages[3005] if isinstance(i, Tag)][0].split()

['https://in.reuters.com/article/iran-economy-rouhani-sanctions/iran-parliament-censures-rouhani-in-sign-pragmatists-losing-sway-idINKCN1LD0DL',
 '[Hassan',
 'Rouhani]',
 'Who',
 'paid',
 'HUSSEIN',
 'to',
 'attend',
 'HARVARD',
 'LAW',
 'SCHOOL?',
 'Who',
 'is',
 'Prince',
 'Alwaleed',
 'bin',
 'Talal?',
 'Why',
 'would',
 'Prince',
 'Alwaleed',
 'bin',
 'Talal',
 '(Saudi',
 'Royal)',
 'pay',
 'HUSSEIN',
 'to',
 'attend',
 'HARVARD',
 'LAW',
 'SCHOOL?',
 'Was',
 'HUSSEIN',
 'a',
 'prominent',
 'political',
 'figure',
 'or',
 'a',
 'person',
 'of',
 'influence',
 'at',
 'the',
 'time?',
 'No.',
 'Who',
 'is',
 'Valerie',
 'Jarrett?',
 'Where',
 'was',
 'she',
 'born?',
 'When',
 'did',
 'Valerie',
 'Jarrett',
 'hire',
 'Michelle',
 'Robinson?',
 '1991',
 'Timeline.',
 'https://www.thisisinsider.com/how-did-barack-and-michelle-obama-meet-2017-10#1991-they-got-engaged-in-a-simple-and-sweet-way-3',
 'Who',
 'is',
 'Mayor',
 '(former)',
 'Richard',
 'Daley?',
 'Who',
 'is',
 'Mayor',
 '(cu

In [74]:
Messages.get(10)

['https://twitter.com/BrentScher/status/1322015793593360384Fact checkers created in effort to reinforce propaganda [digestion]?The battle to prevent truth from reaching the people.The battle to maintain and push division.Divided you are weak.Divided you fight each other.Divided you pose no threat.System of control.Information warfare.Q']

In [29]:
nltk.sent_tokenize(Messages.get(10)[0])

['https://twitter.com/BrentScher/status/1322015793593360384Fact checkers created in effort to reinforce propaganda [digestion]?The battle to prevent truth from reaching the people.The battle to maintain and push division.Divided you are weak.Divided you fight each other.Divided you pose no threat.System of control.Information warfare.Q']

In [13]:
[i.name for i in meta_lar[1].find('span', 'name').children]

['strong', None]

In [14]:
[(i.name, i.attrs) for i in meta_lar[0].children]

[('span', {'class': ['num']}),
 ('span', {'class': ['time']}),
 ('span', {'class': ['name']}),
 ('span', {'class': ['source']}),
 ('span', {'class': ['copy']})]

In [15]:
#drop entire row if name == br?
type(Messages.dataframe(4950))

pandas.core.frame.DataFrame

Now that we have the index taken care of, we can now focus on how to replace

abbr keys with their values in the the dictionary we've been adding too

Remember the are abbrs we want to skip as an exception: POTUS, FBI, HRC, abbrs

whose meaning is obvious and likely won't cloud our analysis of the language 

.index() returns the index of the values first appearance in a list, not all of the indices

In [16]:
for item in Messages.split(4800):
    if item in replace_dict.keys():
        print(item, Messages.split(4800).index(item))

& 182
MSM 216
ID 246
ID 246
SA 290


In [24]:
index_list = [i for i,e in enumerate(Messages.split(4800)) if e in replace_dict.keys()]

In [23]:
[e for i,e in enumerate(Messages.split(4800)) if i == 182]

['&']

In [25]:
[e for i,e in enumerate(Messages.split(4800)) if i in index_list]

['&', 'ID', 'ID']

In [26]:
replace_dict.get('&')

'and'

In [ ]:
Replace an abbr with its value in the replacement_dict: 

for scanner function: 

In [51]:
for item in Messages.split(4000):
    if item in replace_dict.keys():
        item =  replace_dict.get(item)
        print(item)
    else:
        print(item)

How
bad
is
the
corruption?
FBI
(past/present)
#1
#1
#2
+29
(16)
DOJ
(past/present)
#1
#1
#2
+18
STATE
(past/present)
#1
#1
+41
Removal
is
the
least
of
their
problems.
Projection.
Russia>D/
HRC
Twitter
Bots>
GOOG
operated
(not
Russia)/Narrative
and
Political
SLANT
BIDEN
/
CHINA.
BIG
DEVELOPMENT.
TRAITORS
EVERYWHERE.
AMERICA
FOR
SALE.
FLYNN.
Targeted.
Why?
Who
knows
where
the
bodies
are
buried?
CLEARED
OF
ALL
CHARGES.
TRUMP
ADMIN
v2?
Election
theft.
Last
hope.
Congressional
focus.
Impeach.
They
think
you
are
STUPID.
They
think
you
will
follow
the
STARS.
They
openly
call
you
SHEEP/CATTLE.
THERE
WILL
COME
A
TIME
NONE
OF
THEM
WILL
BE
ABLE
TO
WALK
DOWN
THE
STREET.
BIGGEST
FEAR.
PUBLIC
AWAKENING.
Q


In [58]:
Messages.get_abbr(3005)['content'].explode()

59           EU
89     @Snowden
95     @Snowden
103    @Snowden
109         NSA
119          OP
Name: content, dtype: object

this should be the new split logic; what happens otherwise is that the link will be broken up

this way, we can edit it out in one pass 


In [145]:
[Messages.sents(3005)[i].split() for i in range(0, len(Messages.sents(3005)))]

[['https://in.reuters.com/article/iran-economy-rouhani-sanctions/iran-parliament-censures-rouhani-in-sign-pragmatists-losing-sway-idINKCN1LD0DL',
  '[Hassan',
  'Rouhani]',
  'Who',
  'paid',
  'HUSSEIN',
  'to',
  'attend',
  'HARVARD',
  'LAW',
  'SCHOOL?'],
 ['Who', 'is', 'Prince', 'Alwaleed', 'bin', 'Talal?'],
 ['Why',
  'would',
  'Prince',
  'Alwaleed',
  'bin',
  'Talal',
  '(Saudi',
  'Royal)',
  'pay',
  'HUSSEIN',
  'to',
  'attend',
  'HARVARD',
  'LAW',
  'SCHOOL?'],
 ['Was',
  'HUSSEIN',
  'a',
  'prominent',
  'political',
  'figure',
  'or',
  'a',
  'person',
  'of',
  'influence',
  'at',
  'the',
  'time?'],
 ['No.'],
 ['Who', 'is', 'Valerie', 'Jarrett?'],
 ['Where', 'was', 'she', 'born?'],
 ['When', 'did', 'Valerie', 'Jarrett', 'hire', 'Michelle', 'Robinson?'],
 ['1991', 'Timeline.'],
 ['https://www.thisisinsider.com/how-did-barack-and-michelle-obama-meet-2017-10#1991-they-got-engaged-in-a-simple-and-sweet-way-3',
  'Who',
  'is',
  'Mayor',
  '(former)',
  'Richard'

In [ ]:
# This is what the Messages.split() should return, a flat list of words that've been split
# The next step would be to see if any of the items in the flat listen are in replace_dict.keys()

In [127]:
outter_list = []
for items in Messages.split(3005):
    if items.startswith('https'):
        continue
    for char in items:
        if char == '/' or char == '-':
            items = re.sub(char, ' ', items)
    else:
        outter_list.append(items)

In [ ]:
#after the chars are joined back together, if the item is in the replace_dict.keys():
# item = replace_dict.get(item)
# THEN the items can be joined back together to be sent tokenized

In [138]:
nltk.sent_tokenize(' '.join([''.join(i) for i in outter_list]))

['[Hassan Rouhani] Who paid HUSSEIN to attend HARVARD LAW SCHOOL?',
 'Who is Prince Alwaleed bin Talal?',
 'Why would Prince Alwaleed bin Talal (Saudi Royal) pay HUSSEIN to attend HARVARD LAW SCHOOL?',
 'Was HUSSEIN a prominent political figure or a person of influence at the time?',
 'No.',
 'Who is Valerie Jarrett?',
 'Where was she born?',
 'When did Valerie Jarrett hire Michelle Robinson?',
 '1991 Timeline.',
 'Who is Mayor (former) Richard Daley?',
 'Who is Mayor (current) Rahm Emanuel?',
 'HUSSEIN should be VERY nervous.',
 'BRENNAN should be VERY nervous.',
 'KERRY should be VERY nervous.',
 'MERKEL should be VERY nervous.',
 '+29 How were the pallets of cash divided?',
 'How many planes were used to transport?',
 'Who operated the planes?',
 "What 'shadow' agency directed operations?",
 "Why wasn't the money [simply] wire transferred?",
 'US had AUTH to open bank to bank transfers.',
 'How do you prevent financial T logs?',
 'How were the cash withdrawals in EU categorized labe

In [139]:
replace_dict.update({'No Such Agency': 'National Security Agency'})

In [140]:
'U.S.' in replace_dict.keys()

True

In [141]:
"Gov't" in replace_dict.keys()

True

In [11]:
def problem_strings(integer:int):
    item = [e for i,e in enumerate([re.sub(r'https\S+\b', '', item) for item in Messages.split(integer)]) if not e.isalpha()]
    item_idx = [i for i,e in enumerate([re.sub(r'https\S+\b', '', item) for item in Messages.split(integer)]) if not e.isalpha()]
    return list(zip(item_idx, item))

def upper_strings(integer:int):
    item = [e for i,e in enumerate([re.sub(r'https\S+\b', '', item) for item in Messages.split(integer)]) if e.isupper()]
    item_idx = [i for i,e in enumerate([re.sub(r'https\S+\b', '', item) for item in Messages.split(integer)]) if e.isupper()]
    return list(zip(item_idx, item))

In [12]:
problem_strings(3005)

[(0, ''),
 (1, 'Rouhani]Who'),
 (2, 'SCHOOL?Who'),
 (10, 'Talal?Why'),
 (16, '(Saudi'),
 (23, 'Royal)'),
 (24, 'SCHOOL?Was'),
 (31, 'time?No.Who'),
 (45, 'Jarrett?'),
 (46, 'born?When'),
 (50, 'Robinson?1991Timeline.'),
 (54, '(former)'),
 (61, 'Daley?Who'),
 (62, '(current)'),
 (63, 'Emanuel?HUSSEIN'),
 (64, 'nervous.BRENNAN'),
 (68, 'nervous.KERRY'),
 (70, 'nervous.MERKEL'),
 (74, 'nervous.+29How'),
 (76, 'divided?How'),
 (81, 'transport?'),
 (86, 'planes?What'),
 (91, "'shadow'"),
 (96, 'operations?Why'),
 (97, "wasn't"),
 (104, '[simply]'),
 (111, 'transferred?US'),
 (115, 'bank-to-bank'),
 (117, 'transfers.How'),
 (120, 'logs?How'),
 (122, 'categorized/labeled?'),
 (125, 'from?What'),
 (127, 'occur?'),
 (133, 'SECURITY?Why'),
 (134, "wasn't"),
 (141, 'notified?Why'),
 (149, 'U.S.'),
 (155, "Gov't"),
 (163, 'DARK?US'),
 (166, 'broken?Did'),
 (168, '(airport)?'),
 (170, '[1]'),
 (174, 'Iran?Why'),
 (175, 'involved?[WHO]'),
 (179, 'to?HOW'),
 (182, 'TRANSFER?Did'),
 (191, "'unknown'"

In [144]:
def check_string(integer: int):
    length = len(Messages.split(integer))
    
    for s in Messages.split(integer):  
        str_length = [len(s) for s in Messages.split(3005)]
        word_idx = Messages.split(integer).index(s)

        for char in s:           
            for idx in range(0, length):
                char_list = [char for char in Messages.split(integer)[idx] if char in string.punctuation]
                char_idx = [i for i,e in enumerate(Messages.split(integer)[idx]) if e in string.punctuation]

        return str_length

index of characters in the string which are punctuation

In [126]:
[i for i,e in enumerate(Messages.split(3005)[0]) if e in string.punctuation]

[5,
 6,
 7,
 10,
 18,
 22,
 30,
 35,
 43,
 51,
 61,
 66,
 77,
 86,
 94,
 97,
 102,
 114,
 121,
 126]

index and characters on the first string as a tuple

In [107]:
[(i,e) for i,e in enumerate(Messages.split(3005)[0]) if e in string.punctuation]

[(5, ':'),
 (6, '/'),
 (7, '/'),
 (10, '.'),
 (18, '.'),
 (22, '/'),
 (30, '/'),
 (35, '-'),
 (43, '-'),
 (51, '-'),
 (61, '/'),
 (66, '-'),
 (77, '-'),
 (86, '-'),
 (94, '-'),
 (97, '-'),
 (102, '-'),
 (114, '-'),
 (121, '-'),
 (126, '-')]

length of string

In [102]:
len(Messages.split(3005)[0])

140

number of strings

In [103]:
len(Messages.split(3005))

317

In [ ]:
# This scanning can be implented in a function and must occur after hyperlinks have been removed

# Messages.split(integer)[j] for j in range(0, len(Messages.split(integer)))